In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Identity Block

def identity_block(x, filter):
    x_skip = x  # Skip connection

    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3, 3), padding="same")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)

    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3, 3), padding="same")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)

    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation("relu")(x)

    return x

# Convolutional Block (Provided)
def convolutional_block(x, filter):
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3, 3), padding="same", strides=(2, 2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation("relu")(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3, 3), padding="same")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1, 1), strides=(2, 2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation("relu")(x)
    return x

# ResNet34 Architecture
def ResNet34(shape=(224, 224, 3), classes=101):
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)

    # Initial Conv layer and maxPool
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same")(x)

    # ResNet Blocks
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            filter_size *= 2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)

    # Fully Connected Layers
    x = tf.keras.layers.AveragePooling2D((2, 2), padding="same")(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(classes, activation="softmax")(x)

    model = tf.keras.models.Model(inputs=x_input, outputs=x, name="ResNet34")
    return model

# Prepare the Food101 Dataset
def preprocess_data(image, label):
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0  # Normalize
    return image, label

# Load and Preprocess Dataset
def load_data():
    (train_ds, test_ds), ds_info = tfds.load(
        "food101",
        split=["train", "validation"],
        as_supervised=True,
        with_info=True,
    )

    train_ds = train_ds.map(preprocess_data).shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)
    test_ds = test_ds.map(preprocess_data).batch(32).prefetch(tf.data.AUTOTUNE)

    return train_ds, test_ds, ds_info

# Compile and Train the Model
def train_model():
    train_ds, test_ds, ds_info = load_data()
    model = ResNet34(shape=(224, 224, 3), classes=101)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    # Train the Model
    model.fit(
        train_ds,
        validation_data=test_ds,
        epochs=10,  # Set epochs based on your preference
    )

    # Evaluate the Model
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test Accuracy: {test_acc * 100:.2f}%")

if __name__ == "__main__":
    train_model()



2025-01-27 15:33:04.842807: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/workspace/resnet/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
